# About

This notebook demonstrates features of Cling, Xeus-Cling and Jupyter Notebook.

![Project logos](pictures/logos.svg)

# Hello World

In [ ]:
#include <iostream>

std::cout << "Hello World" << std::endl;

- No main() necessary
- each statement is in global space
- some statements are forbidden in global space, like function calls ...
- ... but Cling handles such situations and transforms these statements internally

# Global and Local Variables

In [ ]:
// global variable
int g1 = 1;

In [ ]:
// local variable
{
    int l1 = 2;
}

In [ ]:
std::cout << l1 << std::endl;

In [ ]:
std::cout << g1 << std::endl;
{
    // hide global variable
    int g1 = 3;
    std::cout << g1 << std::endl;
}
std::cout << g1 << std::endl;

# Standard C++ Features

In [ ]:
int fd1(int k){
    return k + 2;
}

In [ ]:
std::cout << fd1(3) << std::endl;

In [ ]:
class Cd1 {
  int a;
  int b;

public:
    Cd1(int a, int b) : a(a), b(b) {}
    int sum(){
        return a + b;
    }
};

In [ ]:
Cd1 cd1(4, 7);
std::cout << cd1.sum() << std::endl;

# Non-Linear Program Flow

In [ ]:
std::cout << non_lin_var << std::endl;

In [ ]:
++non_lin_var;

In [ ]:
int non_lin_var = 3;

# Persistent Memory

In [ ]:
int k = 0;

In [ ]:
for (int end = k + 5; k < end; ++k){
    std::cout << k << std::endl;
}

In [ ]:
k -= 3;

# Template Specialization

In [ ]:
#include <chrono>
constexpr int dim = 512;

In [ ]:
float * A = new float[dim * dim];
float * B = new float[dim * dim];
float * C = new float[dim * dim];

In [ ]:
for(int i = 0; i < dim; ++i){
    A[i] = static_cast<float>(i);
    B[i] = static_cast<float>(i);
}

In [ ]:
void var_matmul(float const * const A, float const * const B, float * const C, const int dim)
{
    float sum = 0.f;
    for (int i = 0; i < dim; ++i) {
      for (int j = 0; j < dim; ++j) {
        for (int k = 0; k < dim; ++k) {
          sum += A[i * dim + k] * B[k * dim + j];
        }
        C[ i * dim + j] = sum;
        sum = 0.f;
      }
    }
}

In [ ]:
template<int dim>
void t_matmul(float const * const A, float const * const B, float * const C)
{
    float sum = 0.f;
    for (int i = 0; i < dim; ++i) {
      for (int j = 0; j < dim; ++j) {
        for (int k = 0; k < dim; ++k) {
          sum += A[i * dim + k] * B[k * dim + j];
        }
        C[ i * dim + j] = sum;
        sum = 0.f;
      }
    }
}

In [ ]:
var_matmul(A, B, C, dim);
t_matmul<dim>(A,B,C);

In [ ]:
{
    std::chrono::time_point<std::chrono::high_resolution_clock> v_start, v_end, t_start, t_end;
    
    v_start = std::chrono::high_resolution_clock::now();
    var_matmul(A, B, C, dim);
    v_end = std::chrono::high_resolution_clock::now();

    t_start = std::chrono::high_resolution_clock::now();
    t_matmul<dim>(A, B, C);
    t_end = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> v_diff = v_end - v_start;
    std::chrono::duration<double> t_diff = t_end - t_start;

    std::cout << "var_matmul: " << v_diff.count() << "s" << std::endl
              << "t_matmul: " << t_diff.count() << "s" << std::endl;
}

# Including and Linking
## Preparation: create a shared library

In [ ]:
%%file foo.hpp
#pragma once

namespace foo {
    int bar();
}

In [ ]:
%%file foo.cpp
#include "foo.hpp"

int foo::bar() { return 42; }

In [ ]:
!gcc -shared foo.cpp -o foo.so

## Call Functionality of the Library

In [ ]:
foo::bar()

In [ ]:
#include "foo.hpp"

In [ ]:
foo::bar()

In [ ]:
#pragma cling(load "foo.so")

In [ ]:
foo::bar()

# REPL Object Representation

In [ ]:
"Hello World"

In [ ]:
int i1 = 3;

In [ ]:
i1

In [ ]:
int fi1(){
    return 42;
}

In [ ]:
fi1()

# Reflection
- values of variables
- type of a variable (cling kernel only)
- memory address
- enum completion (cling kernel only)
- interpreter environment

In [ ]:
struct S {
    int a = 3;
    float b = 6,f;
} s;

In [ ]:
s

In [ ]:
s.a

In [ ]:
#include "cling/Interpreter/Interpreter.h"

In [ ]:
gCling->getDefaultOptLevel()

In [ ]:
gCling->setDefaultOptLevel(3)

# Redefinition

In [ ]:
#include "cling/Interpreter/Interpreter.h"
gCling->allowRedefinition();
gCling->isRedefinitionAllowed();

In [ ]:
int func(){
    return 43;
}

In [ ]:
func()

In [ ]:
class class1 {
    int a = 3;
    int b = 4;
public:
    int func() {
        return a + b;
    }
};

In [ ]:
{
    class1 c;
    std::cout << c.func() << std::endl;
}

# I/O through Web Elements

In [ ]:
#include <string>
#include <fstream>

#include "xtl/xbase64.hpp"
#include "xeus/xjson.hpp"

In [ ]:
void display_image(const std::string filename){
    std::ifstream fin(filename, std::ios::binary);
    std::stringstream buffer;
    buffer << fin.rdbuf();
    // memory objects for output in the web browser
    
    xeus::xjson mine;
    
    xeus::get_interpreter().clear_output(true);
    
    mine["image/png"] = xtl::base64encode(buffer.str());
    xeus::get_interpreter().display_data(
        std::move(mine),
        xeus::xjson::object(),
        xeus::xjson::object());
}

In [ ]:
display_image("pictures/conclusion_basics.png");